In [195]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories
import matplotlib.pyplot as plt
from math import pi

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

In [196]:
eps = 1e-10

center_abs = np.loadtxt("C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Omega.dat",  dtype=float)
v, f = igl.read_triangle_mesh("C:/Users/aikyna/Desktop/hananLab/hJupyter/models/Extra/botanic-garden-parallelogram-0.obj")
V = len(v) # Number of vertices
F = len(f) # Number of faces

v1, v2, k1, k2 = igl.principal_curvature(v, f)
normals = igl.per_vertex_normals(v, f)
h2 = 0.5 * (k1 + k2)

v_bar = np.mean(v[f], axis=1)
normals_per_face = igl.per_face_normals(v, f, np.array([1,1,1]) / np.sqrt(3))
k1_bar = np.mean(k1[f], axis=1)
k2_bar = np.mean(k2[f], axis=1)

rads_min, rads_max = np.zeros(F), np.zeros(F)

# for f_ind in range(F):
#     if (k1_bar[f_ind] != 0):
#         rads_min[f_ind] = 1 / k1_bar[f_ind]
#     else:
#         rads_min[f_ind] = 
#     if (k2_bar[f_ind] != 0):
#         rads_max[f_ind] = 1 / abs(k2_bar[f_ind])
#     if (rads_min[f_ind] > rads_max[f_ind]):
#         rads_min[f_ind], rads_max[f_ind] = rads_max[f_ind], rads_min[f_ind]
        
rads = np.zeros(F)
for f_ind in range(F):
    if ((k1_bar[f_ind] + k2_bar[f_ind]) != 0):
        rads[f_ind] = 1.0 / abs(k1_bar[f_ind] + k2_bar[f_ind])
    else:
        rads[f_ind] = np.inf

centers = np.zeros((F, 3))
for f_ind in range(F):
    if ((k1_bar[f_ind] + k2_bar[f_ind]) != 0):
        centers[f_ind] = v_bar[f_ind] + normals_per_face[f_ind] / (k1_bar[f_ind] + k2_bar[f_ind])

In [197]:
p = mp.plot(v, f, return_plot=True)
p.add_points(np.array([center_abs]), shading={"point_color": "green", "point_size": 1.0})
p.add_lines(v_bar, v_bar + normals_per_face, shading={"line_color": "red"});

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.436749…

In [198]:
rad_init_min = 0
rad_init_max = np.inf
for f_ind in range(F):
    if (rads[f_ind] != np.inf):
        rad_init_min = max(rad_init_min, abs(np.linalg.norm(center_abs - centers[f_ind]) - rads[f_ind]))
        rad_init_max = min(rad_init_max, np.linalg.norm(center_abs - centers[f_ind]) + rads[f_ind])

print("Radius minimum = ", rad_init_min)
print("Radius maximum = ", rad_init_max)

rad_abs = (rad_init_max + rad_init_min) * 0.5

Radius minimum =  18.32786333552335
Radius maximum =  12.876724094729378


In [199]:
J = np.zeros((2 * F, 1 + 2 * F))
r = np.zeros(2 * F)
X_0 = np.concatenate(([rad_abs * rad_abs], np.random.rand(2*F)))

In [200]:
def compute_J(v_bar, f, F, n_face, k1_bar, k2_bar, center_abs, X):
    rad_var = X[0]
        
    for f_ind in range(F):
        k1, k2 = k1_bar[f_ind], k2_bar[f_ind]
        if k1 == 0 and k2 == 0:
            continue
        
        mu_1 = X[2 * f_ind + 1]
        mu_2 = X[2 * f_ind + 2]
        coef = (2 * np.dot(center_abs - v_bar[f_ind], n_face[f_ind]))

        J[f_ind, 0] = 1 / coef        
        J[F + f_ind, 0] = 1 / coef
    
        if abs(k1) > eps:
            if k1 * k2 >= 0:
                rad_max = 1 / k1
                J[f_ind, 2 * f_ind + 1] = np.sign(rad_max) * 2 * mu_1
                r[f_ind] = (rad_var - np.dot(center_abs - v_bar[f_ind], center_abs - v_bar[f_ind])) / coef - rad_max + np.sign(rad_max) * mu_1 ** 2
            else:
                rad_min2 = 1 / k1
                J[f_ind, 2 * f_ind + 1] = -2 * np.sign(rad_min2) * mu_1
                r[f_ind] = (rad_var - np.dot(center_abs - v_bar[f_ind], center_abs - v_bar[f_ind])) / coef - rad_min2 - np.sign(rad_min2) * mu_1 ** 2
        if abs(k2) > eps:
            rad_min = 1 / k2
            J[F + f_ind, 2 * f_ind + 2] = -2 * np.sign(rad_min) * mu_2
            r[F + f_ind] = (rad_var - np.dot(center_abs - v_bar[f_ind], center_abs - v_bar[f_ind])) / coef - rad_min - np.sign(rad_min) * mu_2 ** 2

In [201]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, linalg

# Compute pseudo Hessian
X = X_0

for i in range(15):
    compute_J(v_bar, f, F, normals_per_face, k1_bar, k2_bar, center_abs, X)    

    H = J.T@J

    H[np.diag_indices_from(H)] += np.diag(H).max()*1e-6

    # Sparse matrix H
    H = csc_matrix(H)

    # Solve for dx
    dx = linalg.spsolve(H, -J.T@r)
    # Update vertices
    X = X + 0.9*dx
    
    # energy
    energy = r.T@r
    print(f"energy: {energy}\t dx: {np.linalg.norm(dx)}")
print(X[0])


energy: 303899924.2059997	 dx: 30910.803009141942
energy: 7.684173339073525e+16	 dx: 13893.769392508846
energy: 7031458071927358.0	 dx: 7645.586323906289
energy: 643400272660111.0	 dx: 4207.5987027209985
energy: 58867608436958.32	 dx: 2316.679732816672
energy: 5384432212891.076	 dx: 1285.996348847233
energy: 492042116046.6086	 dx: 777.7744908271962
energy: 46092732787.879295	 dx: 588.3879225019933
energy: 5578752726.659558	 dx: 519.9771872477609
energy: 3175717220.125565	 dx: 514.4932954659104
energy: 2869432768.6253796	 dx: 399.026573619551
energy: 594225549.8826346	 dx: 483.64901323176895
energy: 967328258.4749644	 dx: 443.71561967018675
energy: 664190644.2538393	 dx: 465.29925823777984
energy: 1000695500.470945	 dx: 485.321076605635
-47.81568845666114


In [202]:
rad_abs = 0
if (X[0] < 0):
    rad_abs = np.sqrt(-X[0])
else:
    rad_abs = np.sqrt(X[0])

In [203]:
circ_centers = np.zeros((F, 3))
circ_rads = np.zeros(F)

sphere_centers = np.zeros((F, 3))
sphere_rads = np.zeros(F)

central_centers = np.zeros((F, 3))
central_rads = np.zeros(F)

matrix_a = np.zeros((3,3))
vector_b = np.zeros(3)
matrix_a[0] = np.array([1, 1, 1])
vector_b[0] = 1

count = 0

for f_ind in range(F):
    v0, v1, v2 = v[f[f_ind][0]], v[f[f_ind][1]], v[f[f_ind][2]]
    
    #calculate circumcircle for face
    matrix_a[1] = np.array([-2 * np.dot(v0, v0) + 2 * np.dot(v0, v1), -2 * np.dot(v1,v0) + 2 * np.dot(v1,v1), -2 * np.dot(v2,v0) + 2 * np.dot(v2,v1)])
    vector_b[1] = np.dot(v1, v1) - np.dot(v0, v0)
    matrix_a[2] = np.array([2 * np.dot(v0, v2) - 2 * np.dot(v0, v1), 2 * np.dot(v1,v2) - 2 * np.dot(v1,v1), 2 * np.dot(v2,v2) - 2 * np.dot(v2,v1)])
    vector_b[2] = np.dot(v2, v2) - np.dot(v1, v1)
    mu0, mu1, mu2 = np.linalg.solve(matrix_a, vector_b)
    circ_centers[f_ind] = mu0 * v0 + mu1 * v1 + mu2 * v2
    circ_rads[f_ind] = np.linalg.norm(v0 - circ_centers[f_ind])

    #calculate spheres orthogonal to absolute sphere
    t = (circ_rads[f_ind] ** 2 + X[0] - np.dot(center_abs - circ_centers[f_ind], center_abs - circ_centers[f_ind])) / (2 * np.dot(center_abs - circ_centers[f_ind], normals_per_face[f_ind]))
    sphere_centers[f_ind] = circ_centers[f_ind] - t * normals_per_face[f_ind]
    sphere_rads[f_ind] = np.sqrt(circ_rads[f_ind] ** 2 + t ** 2)

    if (abs(h2[f[f_ind][0]]) < eps or abs(h2[f[f_ind][1]]) < eps or abs(h2[f[f_ind][2]]) < eps):
        print("Problem")
        break
    #calculate central spheres that circumcirscribed to faces
    c0, c1, c2 = v0 - normals[f[f_ind][0]] * 1 / h2[f[f_ind][0]], v1 - normals[f[f_ind][1]] * 1 / h2[f[f_ind][1]], v2 - normals[f[f_ind][2]] * 1 / h2[f[f_ind][2]]
    t = -np.dot(c0 + c1 + c2 - 3 * circ_centers[f_ind], normals_per_face[f_ind])
    central_centers[f_ind] = circ_centers[f_ind] - normals_per_face[f_ind] * t
    central_rads[f_ind] = np.sqrt(circ_rads[f_ind] ** 2 + t ** 2)


In [204]:
#check that spheres circumscribe
for f_ind in range(F):
    v0, v1, v2 = v[f[f_ind][0]], v[f[f_ind][1]], v[f[f_ind][2]] 
    d1, d2, d3 = np.linalg.norm(v0 - sphere_centers[f_ind]), np.linalg.norm(v1 - sphere_centers[f_ind]), np.linalg.norm(v2 - sphere_centers[f_ind])
    r = sphere_rads[f_ind]
    if (abs(r - d1) > eps or abs(r - d2) > eps or abs(r - d3) > eps):
        print("problem")
#check that shperes are orthogonal
for f_ind in range(F):
    m, r = sphere_centers[f_ind], sphere_rads[f_ind]
    if (abs(np.dot(m - center_abs, m - center_abs) - r ** 2 - X[0]) > eps):
        print("problem")

In [205]:
#threshold parameter
delta = 0.1

tol_rads = np.zeros(F)
for f_ind in range(F):
    d = np.linalg.norm(central_centers[f_ind] - sphere_centers[f_ind])
    v1 = sphere_centers[f_ind] - circ_centers[f_ind]
    v2 = central_centers[f_ind] - circ_centers[f_ind]
    v1 /= np.linalg.norm(v1)
    v2 /= np.linalg.norm(v2)
    
    r0 = sphere_rads[f_ind]
    r1, r2 = (central_rads[f_ind] + delta), (central_rads[f_ind] - delta)
    
    if (r2 < 0):
        print("problem")
        break
    a, b = (r0 ** 2 - ((d **2 - r1 ** 2 + r0 ** 2) / 2 / (-d)) ** 2), (r0 ** 2 - ((d **2 - r2 ** 2 + r0 ** 2) / 2 / (-d)) ** 2)
    if r1 > (d + r0):
        a = 0
    if d > (r2 + r0) or r0 > (r2 + d):
        b = 0
    tol_rads[f_ind] = max(np.sqrt(a), np.sqrt(b))
    if (tol_rads[f_ind] == 0):
        print("problem")
        break

In [206]:
ps.init()
ps.remove_all_structures()
 
mesh = ps.register_surface_mesh("Mesh", v, f, smooth_shade=True)
mesh.add_scalar_quantity("Mean curv", h2, defined_on='vertices')

for _ in range(1):
    i = np.random.randint(0, F-1)
    sphere = ps.register_point_cloud(f"sphere_c{i}", np.array([sphere_centers[i]]), enabled=True, color=(0,0,0), transparency=0.3)
    central_sphere = ps.register_point_cloud(f"central_sphere_c{i}", np.array([central_centers[i]]), enabled=True, color=(1,0,0), transparency=0.3)
    sphere.set_radius(sphere_rads[i], relative=False)
    central_sphere.set_radius(central_rads[i], relative=False)

ps.show()